---
author: Nenad Bozinovic
date: 2024-02-01
toc: true
image:  myGPT_thumbnail.jpg
tags: [technical]
title: OpenAI API Demo
---

Try it for yourself at <a href="http://mygpt.nenadbozinovic.com" target="_blank">MyGPT</a>.
Welcome to MyGPT, a simple demo of how to use OpenAI's GPT-4 API.

The design decisions were to use Streamlit for the web app, it is interactive (and WebSocket based) what is for a chat. I use AWS Lambda to call the OpenAI API and the web app is hosted on an EC2 instance (for now). Automatic deployment is done using GitHub Actions.

Main technical points:

## OpenAI client

1. one defines OpenAI client with the API key (pip library is [`openai`](https://pypi.org/project/openai/))

2. chat is a list of messages, each message is a dictionary with `role` (user or bot) and `content` (text). For example this is how we assign a system role, basically a type of assistant you want to have:

```python
chat = [{"role": "system", "content": 'You are software engineer'}]
```

3. To get a reply from the assistant, we use `client.chat.completions.create` method. The model is `gpt` and messages are `chat` list.

```python
def question(chat_history, some_question, client=client):
        """We take a chat_history, append a question as a user, then get a reply from the assistant, and append that too

        Args:
            chat_history (list): A list of dictionaries, with each dictionary containing a role and content key
            some_question (string): 
        """
        chat_history.append({"role": "user", "content": some_question})
        reply = client.chat.completions.create(
                model="gpt-4",
                messages=chat
                )
        reply_message = reply.choices[0].message
        chat_history.append({'role': reply_message.role, 'content':reply_message.content})
        display(Markdown(reply_message.content))
```




We can now test this out:
`question(chat, "What is most popular programming language today?")`

"As of 2021, JavaScript is considered the most popular programming language. However, Python and Java are also widely used and popular. Popularity can depend on the specific sector of software engineering, such as data science (Python) or web development (JavaScript)."
`question(chat, "Can you give me some stats in terms of numbers too?")`

According to the Stack Overflow Developer Survey 2021, the most popular programming languages among professional developers are:

JavaScript - 64.96%  
Python - 44.01%  
Java - 40.2%  
TypeScript - 28.3%  
C# - 27.39%  
PHP - 23.36%  
C++ - 21.57%  
C - 20.59%  

Note that these percentages represent the respondents who use these languages, and as developers often use multiple languages, the percentages will add up to more than 100%. The survey had over 80,000 respondents.
## Call AWS Lambda
We will now move the API calls to cloud, in particular to AWS Lambda. We need to:

- create a layer with the dependencies (I had to use 1.10.12 version of `pydantic` as new one is having issues)
- Lambda has issues with some native libraries per [this](https://docs.aws.amazon.com/lambda/latest/dg/python-package.html#python-package-native-libraries), so make sure to run this when installing packages:
```bash
pip install --platform manylinux2014_x86_64 --target=package --implementation cp --python-version 3.x --only-binary=:all: --upgrade <package_name> -t ./theEnvFolder/python
```
- create a IAM role
- create Lambda function, there are some small tricks of serializing and deserializing the chat history, but it is not a big deal.

We want to pass this chat to a Lambda function and then get back the chat that will have appended the reply from the assistant.
# Lambda function

One can call URL of Lambda using `requests` or create Lambda directly via `boto3`. Having URL might be better since it can be ported to JavaScript down the line. But the two have different `event` variables. I designed Lambda to take care of both. Here is an example using `requests`:

#### `requests`
```python
lambda_url = "xyz"
print("Please wait while GPT4 is done ...")
headers = {'Content-Type': 'application/json'}
r = requests.post(headers=headers, url=lambda_url, json=json.dumps(chat))
response_url = json.loads(r.content.decode('utf-8'))
Markdown(response_url[-1]['content'])
```

Answer: "As of now, JavaScript is considered one of the most popular programming languages due to its wide use in web development. However, other notable languages in popularity include Python, Java, and C#. It can vary depending on the sector and application."
#### `boto3`

```python
lambda_client = boto3.client('lambda')
function_name = 'myGPT'

r = lambda_client.invoke(
    FunctionName=function_name,
    InvocationType='RequestResponse',  # Use 'Event' for asynchronous execution
    Payload=json.dumps(chat))
response_boto3 = json.loads(r['Payload'].read().decode('utf-8'))['body']
Markdown(response_boto3[-1]['content'])
```

As of 2022, the most popular programming language is JavaScript, often followed closely by Python and Java. However, the popularity of a language can vary based on different factors such as the industry, geographical location, and specific tasks or workloads involved.

## Conclusion

We have shown two approaches to call OpenAI API. We can now make a webpage out of this so we can repeatedly call the Lambda function and get the response back.
I have written short [streamlit](https://github.com/nesaboz/blog/blob/main/nbs/projects/myGPT/streamlit.py) code for this. 


We could host this in many ways including third-party options that support WebSockets (as Streamlit needs), I decided to go with my own webapp on EC2.